# 批量读取文件夹中形如 final_solutions_{算法}Seed{种子}.npz 的解集，并将这些解集全部画在同一三维图中
- 不同算法（NSGA2 / NSGA3）用不同颜色区分
- 不同种子用不同的点标记符号区分
- 在合并后的大解集上做一次 非支配排序（ND），再将该 ND 前沿解用同样的颜色与标记画在另外一个三维图中
- 将该大合集中属于 ND 前沿解的 X 与 F 保存到 overall_final_solutions.npz 文件中

## 使用要点说明
- 根据实际的存储路径修改 root_dir。
- 文件命名 pattern = r"final_solutions_(NSGA[23])Seed(\d+)\.npz"。

## 可视化

代码中先后生成了 2 张图：
- 所有解 的三维散点图；
- 合并后 ND 的三维散点图；
- 不同算法用不同颜色，不同 seed 用不同标记；如果有更多算法、更多 seed，可以自行扩展 alg_color_map 和 seed_marker_map。

## 保存 ND 解
- ND 解被以 X_overall, F_overall 的变量名写入到 overall_final_solutions.npz。

In [ ]:
%matplotlib widget

import os
import glob
import re

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting

# 如果需要显示中文，可以添加如下配置 (可选)
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  
matplotlib.rcParams["axes.unicode_minus"] = False

#==============================================================================
# 1. 批量读取所有 final_solutions_*.npz 并区分算法 + 种子
#==============================================================================

# 文件夹路径，示例（请根据实际路径调整）：
root_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\NSGA完整迭代计算结果\0_横向对比"

# 收集 npz 文件：形如 final_solutions_XXX.npz
npz_files = glob.glob(os.path.join(root_dir, "final_solutions_*.npz"))

# 为了在可视化时分配颜色和标记，先做一张“算法->颜色”的映射表:
alg_color_map = {
    "NSGA2": "red",
    "NSGA3": "blue"
}

# 再做一张“种子->marker”映射表（若有更多种子可自行扩展）:
seed_marker_map = {
    1: "o",
    7: "^",
    9: "s",
    10: "D",
    11: "x",
    12: "v",
    13: "<",
    15: ">"
}

# 用于收集全部解
list_all_F = []      # 每个文件的 F
list_all_X = []      # 每个文件的 X
list_all_alg = []    # 该文件属于 NSGA2 或 NSGA3
list_all_seed = []   # 该文件的种子

# 正则匹配：用于从文件名里提取算法和种子，例如 "final_solutions_NSGA2Seed10.npz"
pattern = r"final_solutions_(NSGA[23])Seed(\d+)\.npz"

for filepath in npz_files:
    filename = os.path.basename(filepath)
    
    match = re.match(pattern, filename)
    if not match:
        # 如果不符合匹配，则跳过或提示
        print(f"警告: 文件名 {filename} 不符合形如 final_solutions_{ '{算法}' }Seed{ '{种子}' }.npz 的格式，跳过。")
        continue
    
    # 提取算法与种子
    alg_str = match.group(1)      # "NSGA2" 或 "NSGA3"
    seed_str = match.group(2)     # 例如 "10"
    seed_val = int(seed_str)
    
    # 读取其中的解
    data = np.load(filepath)
    X = data["X"]  # (n_points, n_var)
    F = data["F"]  # (n_points, n_obj)
    
    # 收集
    list_all_F.append(F)
    list_all_X.append(X)
    list_all_alg.extend([alg_str]*len(F))
    list_all_seed.extend([seed_val]*len(F))

# 把列表拼成大数组
if len(list_all_F) == 0:
    raise ValueError("没有找到任何符合要求的 npz 文件，程序退出。")

F_all = np.vstack(list_all_F)  # shape = (N_total, n_obj=3)
X_all = np.vstack(list_all_X)  # shape = (N_total, n_var=?)

# 转成 np.array 方便后面索引
all_alg = np.array(list_all_alg)    # shape = (N_total,)
all_seed = np.array(list_all_seed)  # shape = (N_total,)

#------------------------------------------------------------------------------
# 2. 绘制所有点的 3D 图
#    - 不同算法用不同颜色
#    - 不同种子用不同 marker
#------------------------------------------------------------------------------

fig_all = plt.figure(figsize=(8, 6))
ax_all = fig_all.add_subplot(111, projection='3d')

# 为了只在图例里显示一次对应的 label，给 (alg, seed) 做一个去重控制
plotted_labels = set()

for i in range(len(F_all)):
    alg_i = all_alg[i]   # "NSGA2" or "NSGA3"
    seed_i = all_seed[i]
    
    # 取颜色、标记
    color_i = alg_color_map.get(alg_i, "black")     # 如果没匹配上，默认 black
    marker_i = seed_marker_map.get(seed_i, ".")     # 如果没匹配上，默认 '.'
    
    # 做一个 (alg, seed) 的组合，用来控制只在第一次出现时加 label
    label_key = (alg_i, seed_i)
    if label_key not in plotted_labels:
        label_str = f"{alg_i}, seed={seed_i}"
        plotted_labels.add(label_key)
    else:
        label_str = None  # 避免重复显示 legend
    
    # 画散点
    ax_all.scatter(
        F_all[i, 0], F_all[i, 1], F_all[i, 2],
        c=color_i, marker=marker_i, s=40,
        label=label_str
    )

ax_all.set_xlabel("F0")
ax_all.set_ylabel("F1")
ax_all.set_zlabel("F2")
ax_all.set_title("所有解点的三维分布: 不同算法(颜色), 不同种子(marker)")
ax_all.legend()
plt.tight_layout()
plt.show()


#------------------------------------------------------------------------------
# 3. 对合并后的所有解做一次新的非支配排序，并可视化 ND 前沿解
#------------------------------------------------------------------------------
sorting = NonDominatedSorting().do(F_all, only_non_dominated_front=True)
F_nd = F_all[sorting]
X_nd = X_all[sorting]
alg_nd = all_alg[sorting]
seed_nd = all_seed[sorting]

fig_nd = plt.figure(figsize=(8, 6))
ax_nd = fig_nd.add_subplot(111, projection='3d')

plotted_labels_nd = set()

for i in range(len(F_nd)):
    alg_i = alg_nd[i]
    seed_i = seed_nd[i]
    
    color_i = alg_color_map.get(alg_i, "black")
    marker_i = seed_marker_map.get(seed_i, ".")
    
    label_key = (alg_i, seed_i)
    if label_key not in plotted_labels_nd:
        label_str = f"{alg_i}, seed={seed_i}"
        plotted_labels_nd.add(label_key)
    else:
        label_str = None
    
    ax_nd.scatter(
        F_nd[i, 0], F_nd[i, 1], F_nd[i, 2],
        c=color_i, marker=marker_i, s=40,
        label=label_str
    )

ax_nd.set_xlabel("F0")
ax_nd.set_ylabel("F1")
ax_nd.set_zlabel("F2")
ax_nd.set_title("合并后非支配前沿解 (ND) 的三维分布")
ax_nd.legend()
plt.tight_layout()
plt.show()


#------------------------------------------------------------------------------
# 4. 将该大合集中属于 ND 前沿解的 X, F 保存
#------------------------------------------------------------------------------
output_path = os.path.join(root_dir, "Overall_FinalSolutions.npz")
np.savez(output_path, final_X = X_nd, final_F = F_nd)
print(f"[Info] 已保存合并ND前沿解到: {output_path}")


# plotly 交互式 3D 散点图

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import numpy as np

# 假设已有数组 F_nd, alg_nd, seed_nd
# F_nd.shape = (562, 3), alg_nd.shape = (562, ), seed_nd.shape = (562, )

# 为了可以在同一个散点图里用颜色/符号区分
#   - 颜色: 不同算法
#   - marker 符号: 不同种子
#
# 可以将数组 alg_nd、seed_nd 转成字符串形式，用于区分

df_dict = {
    "F0": F_nd[:, 0],
    "F1": F_nd[:, 1],
    "F2": F_nd[:, 2],
    "Algorithm": alg_nd,
    "Seed": seed_nd
}

import pandas as pd
df = pd.DataFrame(df_dict)

# 使用 Plotly Express 可以快速生成 3D 散点图
fig = px.scatter_3d(
    df,
    x="F0", y="F1", z="F2",
    color="Algorithm",          # 按算法分颜色
    symbol="Seed",              # 按种子分 marker symbol
    opacity=0.8,                # 点透明度
    title="合并后非支配前沿解: Plotly交互"
)

fig.update_layout(
    width = 1000,
    height = 1000,
    legend=dict(x=0.01, y=0.99)  # 调整图例位置
)

# 这里将所有散点 marker 的 size 设置为 3
fig.update_traces(marker=dict(size=5))

fig.show()


# 从 Result_Sweep_manyNSGARlts 中提取三个指标并作图
**用 NonDominatedSorting 做非支配排序，得到整个数据集的 非支配前沿（ND Front）**

- 第 0 号指标（临界速度，越大越好）
- 第 1 号指标（曲线磨耗数，越小越好）
- 第 5 号指标（平稳性指标-SperlingY，越小越好）
- 在一个 3D 图里显示 所有点 和 ND 前沿点，并且在坐标轴上显示真实指标值

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
import os

# 如果需要中文显示，可根据系统配置选用字体(可选)
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]
matplotlib.rcParams["axes.unicode_minus"] = False

# 1. 加载数据
root_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\NSGA完整迭代计算结果\0_横向对比"
filepath = os.path.join(root_dir, "Result_Sweep_manyNSGARlts.npy")

data = np.load(filepath)  # shape = (7, 562)
# 转置，使之变成 (562, 7)，行代表每个悬挂组合，列代表7个指标
data_T = data.T

# 2. 提取关心的三个指标: idx=0,1,5
#    - 0: 临界速度 (越大越好)
#    - 1: 曲线磨耗数 (越小越好)
#    - 5: SperlingY (越小越好)
#    对于 ND 排序，需要全部指标都是 "越小越好" => 对 idx=0 的指标取负号

# 用于非支配排序 (转化后全是“越小越好”)
F_sort = np.zeros((data_T.shape[0], 3))
F_sort[:, 0] = -data_T[:, 0]  # 取负，临界速度越大越好 -> (取负) -> 越小越好
F_sort[:, 1] =  data_T[:, 1]  # 磨耗数越小越好 -> 保持
F_sort[:, 2] =  data_T[:, 5]  # SperlingY 越小越好 -> 保持

# 同时，做一个“可视化”用的数据 (不取负, 保持原值, 便于阅读)
F_display = np.column_stack([
    data_T[:, 0],  # 临界速度 (实际数值, 正)
    data_T[:, 1],  # 磨耗数
    data_T[:, 5]   # SperlingY
])

# 3. 非支配排序
nds = NonDominatedSorting().do(F_sort, only_non_dominated_front=True)

# 取得 ND 前沿
F_nd_sort    = F_sort[nds]      # 用于检查 ND 的真实坐标(都是越小越好形式)
F_nd_display = F_display[nds]   # 用于可视化时保持原值

# 4. 画 3D 图

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

# # (a) 绘制所有点 (灰色, alpha=0.1)
# ax.scatter(
#     F_display[:,0], F_display[:,1], F_display[:,2],
#     c='gray', alpha=0.3, marker='o', s=20,
#     label='All 562 combos',
#     depthshade=False
# )

# (b) 绘制 ND 前沿 (红色, alpha=1.0)
ax.scatter(
    F_nd_display[:,0], F_nd_display[:,1], F_nd_display[:,2],
    c='red', alpha=1.0, marker='^', s=40,
    label='ND Front',
    depthshade=False
)

ax.set_xlabel("临界速度 (越大越好)")
ax.set_ylabel("磨耗数 (越小越好)")
ax.set_zlabel("SperlingY (越小越好)")
ax.set_title("三个指标的非支配前沿 (ND)")
ax.legend()
# 让 y 轴反向显示（把磨耗数的坐标翻转）
ax.invert_yaxis()
plt.tight_layout()
plt.show()


# 刚性轮对曲线通过磨耗数 - IRW曲线通过磨耗数关系
F_NativeRigidWN =  data_T[:, 1]  # 刚性轮对 磨耗数 
F_CtrlIRWWN =  data_T[:, 3]  # IRW轮对 磨耗数

plt.figure(figsize=(6, 5))
plt.scatter(F_NativeRigidWN, F_CtrlIRWWN, c='b', alpha=0.6, edgecolors='k',
            label='刚性 vs. IRW 散点')
plt.legend()
plt.xlabel("刚性轮对 磨耗数")
plt.ylabel("IRW轮对 磨耗数")
plt.title("散点图：刚性轮对 vs. IRW轮对")
plt.grid(True)
plt.tight_layout()
plt.show()

# 占位

In [ ]:
# 代码占位